# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data.

The dataset we'll be using today is located in the lab directory named `Sentiment140.csv.zip`. You need to unzip it into a `.csv` file. Then in the cell below, load and explore the data.

*Notes:* 

* The dataset was downloaded from [Kaggle](https://www.kaggle.com/kazanova/sentiment140). We made a slight change on the original data so that each column has a label.

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [30]:
import pandas as pd
import re
import nltk
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords

In [31]:
# your code here
df = pd.read_csv('../data/twitter.csv', encoding='latin-1')

In [32]:
tw_sample = df.sample(20000)
tw_sample.columns

Index(['0', '1467810369', 'Mon Apr 06 22:19:45 PDT 2009', 'NO_QUERY',
       '_TheSpecialOne_',
       '@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D'],
      dtype='object')

In [33]:
tw_sample = tw_sample.rename(columns={'0': 'target', '1467810369': 'id', 'Mon Apr 06 22:19:45 PDT 2009': 'date', 'NO_QUERY': 'flag', '_TheSpecialOne_': 'user', "@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D": 'tweet'})


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [34]:
def clean_up(s):
    """
    Cleans up numbers, URLs, and special characters from a string.

    Args:
        s: The string to be cleaned up.

    Returns:
        A string that has been cleaned up.
    """
    without_url = re.sub(r'(?=\ ht).*$', '', s) #picks all until ht
    just_letters = re.findall(r'[^\W\d_]+', without_url.lower()) # find words without special characters and lowering the string
    return just_letters

def tokenize(s):
    """
    Tokenize a string.

    Args:
        s: String to be tokenized.

    Returns:
        A list of words as the result of tokenization.
    """
    return word_tokenize(s)

def stem_and_lemmatize(l):
    """
    Perform stemming and lemmatization on a list of words.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after being stemmed and lemmatized.
    """
    lemmatize_words = [lemmatizer.lemmatize(i) for i in l]
    return lemmatize_words

def remove_stopwords(l):
    """
    Remove English stopwords from a list of strings.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after stop words are removed.
    """
    without_stopwords = []
    for i in l:
        if i not in stopwords.words('english'):
            without_stopwords.append(i)
    return without_stopwords

In [35]:
tw_sample['text_processed'] = tw_sample['tweet'].apply(clean_up)
#tw_sample['text_processed'] = tw_sample['text_processed'].apply(nltk.word_tokenize)
tw_sample['text_processed'] = tw_sample['text_processed'].apply(stem_and_lemmatize)
tw_sample['text_processed'] = tw_sample['text_processed'].apply(remove_stopwords)

In [36]:
tw_sample

,target,id,date,flag,user,tweet,text_processed
241175,0,1981037142,Sun May 31 08:16:12 PDT 2009,NO_QUERY,chaz_6,#Pidgin 2.5.6 + #GnuTLS 2.8.0 = #gtalk woes (T...,"[pidgin, gnutls, gtalk, woe, tl, packet, unexp..."
1237966,4,1993144195,Mon Jun 01 09:54:11 PDT 2009,NO_QUERY,Fatinnn,"@lemonn Oh, okay. All the best to you!","[lemonn, oh, okay, best]"
867717,4,1677825373,Sat May 02 04:11:22 PDT 2009,NO_QUERY,twebbstack,@shaundiviney HAHHAHAH lols. lucky im not inf...,"[shaundiviney, hahhahah, lols, lucky, im, infe..."
886730,4,1686802686,Sun May 03 06:49:26 PDT 2009,NO_QUERY,KindlySmiling,Came back from Mass..... now im tired...!,"[came, back, mass, im, tired]"
817403,4,1551746678,Sat Apr 18 09:52:05 PDT 2009,NO_QUERY,christyitamoto,@ChristopherY Mornin to ya! You have a great w...,"[christophery, mornin, ya, great, weekend]"
...,...,...,...,...,...,...,...
208254,0,1973679669,Sat May 30 11:37:22 PDT 2009,NO_QUERY,mariavittoriaa,@brenddaxo lmao youu do love boootay. Andd fml...,"[brenddaxo, lmao, youu, love, boootay, andd, f..."
1099494,4,1970578894,Sat May 30 04:25:44 PDT 2009,NO_QUERY,MelizzaLykk,Having a needed fag in my garden. Cba with any...,"[needed, fag, garden, cba, anything, today, im..."
728764,0,2263151941,Sun Jun 21 00:04:47 PDT 2009,NO_QUERY,hisydneyxo,@lovekelsey same here...IM WIDE AWAKE. no one ...,"[lovekelsey, im, wide, awake, one, sleep, bet,..."
1403752,4,2054919628,Sat Jun 06 08:20:44 PDT 2009,NO_QUERY,ellefff,@delicsy ahaha well expected la. she hangs aro...,"[delicsy, ahaha, well, expected, la, hang, aro..."


In [37]:
# your code here
tw_sample['text_processed']

241175     [pidgin, gnutls, gtalk, woe, tl, packet, unexp...
1237966                             [lemonn, oh, okay, best]
867717     [shaundiviney, hahhahah, lols, lucky, im, infe...
886730                         [came, back, mass, im, tired]
817403            [christophery, mornin, ya, great, weekend]
                                 ...                        
208254     [brenddaxo, lmao, youu, love, boootay, andd, f...
1099494    [needed, fag, garden, cba, anything, today, im...
728764     [lovekelsey, im, wide, awake, one, sleep, bet,...
1403752    [delicsy, ahaha, well, expected, la, hang, aro...
468841                                     [dodgy, follower]
Name: text_processed, Length: 20000, dtype: object

### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [38]:
words_in_tweet = []
for lists in tw_sample['text_processed']:
    for word in lists:
        words_in_tweet.append(word)

In [39]:
# your code here
from nltk.probability import FreqDist
word_dist = nltk.FreqDist(words_in_tweet)

In [40]:
top_5000 = word_dist.most_common(5000)

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [41]:
# your code here
bag_words = [i[0] for i in top_5000]

In [63]:
#words = tw_sample['text_processed']
features = {}
for w in bag_words:
    for words in tw_sample['text_processed']:
        features[w] = (w in words)

In [69]:
features

{'day': False,
 'wa': False,
 'good': False,
 'get': False,
 'like': False,
 'go': False,
 'today': False,
 'u': False,
 'work': False,
 'love': False,
 'quot': False,
 'time': False,
 'going': False,
 'got': False,
 'one': False,
 'lol': False,
 'know': False,
 'im': False,
 'back': False,
 'amp': False,
 'really': False,
 'want': False,
 'see': False,
 'night': False,
 'think': False,
 'home': False,
 'still': False,
 'miss': False,
 'new': False,
 'need': False,
 'oh': False,
 'ha': False,
 'well': False,
 'thanks': False,
 'much': False,
 'make': False,
 'last': False,
 'morning': False,
 'tomorrow': False,
 'great': False,
 'feel': False,
 'hope': False,
 'twitter': False,
 'haha': False,
 'wish': False,
 'fun': False,
 'right': False,
 'sleep': False,
 'sad': False,
 'happy': False,
 'bad': False,
 'come': False,
 'nice': False,
 'thing': False,
 'week': False,
 'tonight': False,
 'would': False,
 'friend': False,
 'sorry': False,
 'wait': False,
 'way': False,
 'look': False,
 '

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [70]:
# your code here
training = features[:1500]
test = features[1500:]
from nltk.classify.api import ClassifierI
classifier = nltk.NaiveBayesClassifier.train
classifier.show_most_informative_features(10)

TypeError: unhashable type: 'slice'

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [ ]:
# your code here

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here